In [43]:
import os
from dotenv import load_dotenv
from IPython.display import JSON
import ccxt
import logging
from math import ceil

logger = logging.getLogger()
logger.setLevel(logging.INFO)

load_dotenv()

True

In [20]:
class Portfolio:
    def __init__(self, ratios, total_buy_volume_krw):
        self.total_buy_volume_krw = total_buy_volume_krw
        
        self.portfolio = {}
        for symbol, ratio in ratios.items():
            self.portfolio[symbol] = {
                'symbol': symbol,
                'ratio' : ratio,
                'buy_volume_krw': ratio * total_buy_volume_krw
            }

    def __iter__(self):
        for val in self.portfolio.values():
            yield val


In [38]:
ratios = {
    'BTC/KRW': 0.5,
    'ETH/KRW': 0.3,
    'SOL/KRW': 0.1,
    'LINK/KRW': 0.1
}

portfolio = Portfolio(ratios, 50000)

In [39]:
bithumb = ccxt.bithumb(config={
    'apiKey': os.getenv("ACCESS_KEY"),
    'secret': os.getenv("SECRET_KEY"),
    'enableRateLimit': True
})
bithumb.fetch_balance()['KRW']

{'free': 300572.0, 'used': 0.0, 'total': 300572.0}

In [45]:
for coin in portfolio:
    amount = coin['buy_volume_krw'] / bithumb.fetch_ticker(coin['symbol'])["close"]
    amount = ceil(amount * 1000000) / 1000000.0
    logging.info(f'''  Symbol: {coin['symbol']}, Ratio: {coin['ratio']}, Buy_Amount_KRW: {coin['buy_volume_krw']} Buy_Amount_Symbol: { amount }''')
    
    
    resp = bithumb.create_market_buy_order(
        symbol=coin['symbol'],
        amount=amount
    )
    
    logging.info(resp)

INFO:root:  Symbol: BTC/KRW, Ratio: 0.5, Buy_Amount_KRW: 25000.0 Buy_Amount_Symbol: 0.000163
INFO:root:{'info': {'status': '0000', 'order_id': 'C0101000002336292159'}, 'symbol': 'BTC/KRW', 'type': 'market', 'side': 'buy', 'id': 'C0101000002336292159', 'fees': [], 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'price': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'timeInForce': 'IOC', 'postOnly': None, 'trades': [], 'reduceOnly': None, 'stopPrice': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'status': None, 'fee': None}
INFO:root:  Symbol: ETH/KRW, Ratio: 0.3, Buy_Amount_KRW: 15000.0 Buy_Amount_Symbol: 0.004057
INFO:root:{'info': {'status': '0000', 'order_id': 'C0102000001084566431'}, 'symbol': 'ETH/KRW', 'type': 'market', 'side': 'buy', 'id': 'C0102000001084566431', 'fees': [], 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'lastT